# HASHING DE FEATURES
---
O conjunto de dados é estritamente categórico possuindo alta cardinalidade, estando sujeito também a adição e remoção de novos perfis e atribuições a qualquer momento.

Este trabalho objetiva construir um recomendador de perfis que:

1 - Não requeira qualquer tipo de ajuste manual dos dados processados quando ocorrerem modificações no conjunto de perfis existente e de suas atribuições.

2 - Consiga lidar com uma ordem de magnitude a mais dos perfis existentes (de milhares para milhões), pois isto possibilitaria realizar a recomendação de, além de perfis dos SAP, também de direitos controlados pelo IdentityIQ e grupos do Active Directory.

Tomado em conjunto os requisitos acima, constatou-se que a técnica conhecida como feature hashing associada ao uso de shingling atende ao cenário proposto.


In [1]:
import numpy as np
import os
import duckdb
import pandas as pd
import hashlib
import math
import scipy.sparse
import bpe
from unidecode import unidecode
from pprint import pprint

As variáveis de ambiente abaixo precisam ser configuradas antes da execução deste notebook. Vide o arquivo **setenv.ps1.example**

In [2]:
DATASET                     = os.environ['DATASET']
HASHED_FEATURE_COUNT        = int(os.environ['HASHED_FEATURE_COUNT'])
HASHED_FEATURES             = os.environ['HASHED_FEATURES']
HASHED_FEATURES_IDX         = os.environ["HASHED_FEATURES_IDX"]

Esta é uma lista de stop tokens que o processo extração de features de variáveis textuais irá ignorar. Ademais, somente serão considerados tokens com MIN_TOKEN_COUNT caracteres

In [3]:
MIN_TOKEN_COUNT = 4
STOP_TOKENS = set([
    # remove tokens de controle do encoder
    '__eow', '__sow', '__unk', '__pad'
])

def filter_tokens(tokens):
    for token in tokens:
        if len(token) < 4 or token in STOP_TOKENS:
            continue
        yield token

Leitura do conjunto de dados contendo as informações da base integrada de RH e as atribuições de perfis

In [4]:
dataset_df = pd.read_parquet(DATASET)
dataset_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91798 entries, 0 to 91797
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   chave_usuario            91798 non-null  object
 1   tipo_usuario             91798 non-null  object
 2   centro_custo             39913 non-null  object
 3   lotacao_topo             91798 non-null  object
 4   sigla_lotacao            91798 non-null  object
 5   nome_lotacao             91798 non-null  object
 6   cargo                    91798 non-null  object
 7   enfase                   91798 non-null  object
 8   funcao                   91798 non-null  object
 9   sindicato                39415 non-null  object
 10  area_rh                  91264 non-null  object
 11  imovel                   91264 non-null  object
 12  local_negocio            91261 non-null  object
 13  grupo_prestacao_servico  51118 non-null  object
 14  regime_trabalho          76638 non-nul

In [5]:
def clean_text(s):
    return unidecode(s).lower()

def iternestedvalues(dataset_df, property_name):
    for objs in dataset_df[property_name]:
        for obj in objs:
            yield obj

In [6]:
siglas        = dataset_df['sigla_lotacao'].tolist()
siglas_set    = set(siglas)
sigla_encoder = bpe.Encoder()
sigla_encoder.fit(siglas)
list(sigla_encoder.tokenize('TIC/CORP/RJTI/PN-CI'))

['tic', '/', 'corp', '/', 'rjti', '/', 'pn', '-', 'ci']

In [7]:
nomes_lotacao         = dataset_df['nome_lotacao'].tolist()
nome_lotacao_encoder = bpe.Encoder()
nome_lotacao_encoder.fit(nomes_lotacao)
list(nome_lotacao_encoder.tokenize('GESTAO DA DISPONIBILIDADE DE RECURSOS E SERVICOS'))

['gestao', 'da', 'disponibilidade', 'de', 'recursos', 'e', 'servicos']

In [8]:
cargos = dataset_df['cargo'].tolist()
cargo_encoder = bpe.Encoder()
cargo_encoder.fit(cargos)
list(cargo_encoder.tokenize('ASSISTENTE TECNICO EXECUTIVO DIRETORIA'))

['assistente', 'tecnico', 'executivo', 'diretoria']

In [9]:
enfases = dataset_df['enfase'].tolist()
enfase_encoder = bpe.Encoder()
enfase_encoder.fit(enfases)
list(enfase_encoder.tokenize('PCR NT SUPRIMENTO DE BENS E SERVICOS MECANICA'))

['pcr', 'nt', 'suprimento', 'de', 'bens', 'e', 'servicos', 'mecanica']

In [10]:
funcoes = dataset_df['funcao'].tolist()
funcao_encoder = bpe.Encoder()
funcao_encoder.fit(funcoes)
list(funcao_encoder.tokenize('COORDENADOR(A) DE TURNO'))

['coordenador', '(', 'a', ')', 'de', 'turno']

In [11]:
empresas_contrato = dataset_df['empresa_contrato'].fillna('n/a').tolist()
empresa_contrato_encoder = bpe.Encoder()
empresa_contrato_encoder.fit(empresas_contrato)
list(empresa_contrato_encoder.tokenize('MKM RIO PRESTACAO DE SERVICOS LTDA ME'))

['mkm', 'rio', 'prestacao', 'de', 'servicos', 'ltda', 'me']

In [12]:
objetos         = list(iternestedvalues(dataset_df, 'objetos_contratos'))
objetos_set     = set(objetos)
objetos_encoder = bpe.Encoder()
objetos_encoder.fit(objetos)
list(objetos_encoder.tokenize('LICENCIAMENTO DE USO, MANUTENÇÃO E SUPORTE TÉCNICO DOS SOFTWARES E-TRAC E SECURSEARCH'))

['licenciamento',
 'de',
 'uso',
 ',',
 'manutenção',
 'e',
 'suporte',
 'técnico',
 'dos',
 'softwares',
 'e',
 '-',
 '__sow',
 'tr',
 'ac',
 '__eow',
 'e',
 '__sow',
 'se',
 'cu',
 'rs',
 'ea',
 'rc',
 'h',
 '__eow']

In [13]:
cursos         = list(iternestedvalues(dataset_df, 'cursos'))
cursos_set     = set(cursos)
cursos_encoder = bpe.Encoder()
cursos_encoder.fit(cursos)
list(cursos_encoder.tokenize("'TM - LISTAS DE OBJETOS DE TRANSPORTE [SAPTMSTMP15]"))

["'",
 'tm',
 '-',
 'listas',
 'de',
 'objetos',
 'de',
 'transporte',
 '[',
 'saptmstmp15',
 ']']

In [14]:
roles         = list(iternestedvalues(dataset_df, 'roles'))
roles_set     = set(roles)
roles_encoder = bpe.Encoder()
roles_encoder.fit(roles)
list(roles_encoder.tokenize("Z:FI_AA_PB001_EXE_CON_REL_IMOB"))

['z',
 ':',
 '__sow',
 'fi',
 '_a',
 'a_',
 'pb',
 '00',
 '1_',
 'ex',
 'e_',
 'co',
 'n_',
 're',
 'l_',
 'im',
 'ob',
 '__eow']

Abaixo, segue as configurações de que campos serão processados usando a técnica de *shingling*.

Constatamos que um mecanismo de pesos para os diferentes atributos pode eventualmente ser útil para realização do ajuste fino do classificador, mas dado a natureza exploratória desse trabalho, este mecanismo não foi utilizado.

In [15]:
EXCLUDED_FEATURES           = set(['chave_usuario',])
DEFAULT_FEATURE_WEIGHT      = 1
FEATURE_WEIGHTS             = {'sigla_lotacao' : 3, 'cursos': 0.125, 'objetos_contratos': 0.25}
TOKENIZERS                  = {
    'sigla_lotacao'         : sigla_encoder
,   'nome_lotacao'          : nome_lotacao_encoder
,   'objetos_contratos'     : objetos_encoder
,   'cursos'                : cursos_encoder
,   'roles'                 : roles_encoder
,   'nome_lotacao'          : nome_lotacao_encoder 
,   'cargo'                 : cargo_encoder
,   'enfase'                : enfase_encoder
,   'funcao'                : funcao_encoder
,   'empresa_contrato'      : empresa_contrato_encoder
}

In [16]:
print(cargo_encoder.tokenize("ANALISTA DE SISTEMAS SENIOR"))
print(cargo_encoder.tokenize("ANALISTA DE SISTEMAS PLENO"))
print(cargo_encoder.tokenize("INSPETOR(A) DE SEGURANCA INTERNA PLENO"))

['analista', 'de', 'sistemas', 'senior']
['analista', 'de', 'sistemas', 'pleno']
['inspetor', '(', 'a', ')', 'de', 'seguranca', 'interna', 'pleno']


In [17]:
def hash_string(s):
    buf        = str(s).encode()
    sha1       = hashlib.sha1(buf)
    md5        = hashlib.md5(buf)
    hash_value = int(sha1.hexdigest(), base=16)
    sign_value = int(md5.hexdigest(), base=16)
    sign       = -1 if abs(sign_value % 2) == 1 else 1 
    result     = sign * hash_value 
    return result

Abaixo, calculamos o hash de um feature categórico. O feature é codificado como **"&lt;nome do feature&gt;::&lt;valor do feature&gt;"**

In [18]:
n = 64
valor_original = "sigla_lotacao::TIC/CORP/DSCESI/DS-PDDS"
hash_lotacao = hash_string(valor_original)
hash_original_mod_n = abs(hash_lotacao) % n 
print(f"""
valor                  : {valor_original}
hash_lotacao           : {hash_lotacao}
abs(hash_lotacao) % {n} : {hash_original_mod_n}
""")


valor                  : sigla_lotacao::TIC/CORP/DSCESI/DS-PDDS
hash_lotacao           : 580521354414209253383885614425930233875981244812
abs(hash_lotacao) % 64 : 12



Abaixo, calculamos uma alteração em um feature que provoque uma colisão com o primeiro hash, mas não do segundo 

In [19]:
valor_inicial = "tipo_usuario::PRESTADOR DE SERVIÇO"
hash_colisao = None
chs = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!?.,"
for ch1 in chs:
    if hash_colisao is not None:
        break
    for ch2 in chs:
        if hash_colisao is not None:
            break
        for ch3 in chs:
            valor_final = valor_inicial + ch1 + ch2 + ch3
            hash = hash_string(valor_final)
            if hash < 0 and abs(hash) % n == hash_original_mod_n:
                hash_colisao = hash
                break
print(f"""
valor                  : {valor_final}
hash_colisao           : {hash_colisao}
abs(hash_colisao) % {n} : {abs(hash_colisao) % n}
""")


valor                  : tipo_usuario::PRESTADOR DE SERVIÇOafk
hash_colisao           : -152937679684757113643692380177006773294535406668
abs(hash_colisao) % 64 : 12



Neste cenário fictício, ambas as features seriam mapeadas para a posição 12 de um vetor de tamanho 16 (tamanho arbitrariamente reduzido para provocar colisão neste cenário artificial).

Neste caso, o efeito da colisão é mitigado pela diferença de sinais. Vide detalhes na parte teórica do trabalho.

Segue abaixo as funções utilizado para realizar o feature hashing com e sem *shingling*

In [20]:
def hash_feature(name, value, p=DEFAULT_FEATURE_WEIGHT):
    if value is None or value == "" or value == 0:
        return []
    feature = f"{name}::{value}"
    return feature, hash_string(feature), p

In [21]:
feature, hashes, p = hash_feature("sigla_lotacao", "TIC/CORP/DSCESI/DS-PDDS")
print(f"""
feature : {feature}
hashes  : {hashes}
p       : {p}
""")


feature : sigla_lotacao::TIC/CORP/DSCESI/DS-PDDS
hashes  : 580521354414209253383885614425930233875981244812
p       : 1



In [22]:
feature, hashes, p = hash_feature("tipo_usuario", "PRESTADOR DE SERVIÇOafk")
print(f"""
feature : {feature}
hashes  : {hashes}
p       : {p}
""")


feature : tipo_usuario::PRESTADOR DE SERVIÇOafk
hashes  : -152937679684757113643692380177006773294535406668
p       : 1



In [23]:
def hash_tokens(tokenizer, name, value, p):
    if value is None or value == "" or value == 0:
        return []
    tokens    = list(filter_tokens(tokenizer.tokenize(value)))
    hashes    = []
    features  = []
    for token in tokens:
        if len(token) < 4 or token in STOP_TOKENS:
            continue
        feature = f"{name}::{token}"
        h = hash_string(feature)
        features.append(feature)
        hashes.append(h)
    
    token_count = len(tokens)
    if token_count == 0:
        return []
    total_tokens_size = sum([len(token) for token in tokens])
    ps = [ len(token) / total_tokens_size for token in tokens ]    
    return list(zip(features, hashes, ps))

In [24]:
list(hash_tokens(sigla_encoder, 'sigla_lotacao', "TIC/CORP/DSCESI/DS-PDDS", 3))

[('sigla_lotacao::corp',
  1195458149619987768667806989226000270421411538951,
  0.2857142857142857),
 ('sigla_lotacao::dscesi',
  425021426705904005561851342121354676834907806866,
  0.42857142857142855),
 ('sigla_lotacao::pdds',
  133469625148378281707186843726286609828382815863,
  0.2857142857142857)]

A função abaixo é a funçaõ que recebe um dicionário contendo os dados dos usuários assim como uma lista de roles (como um atributo multi-valorado) e retorna uma lista de tuplas no formato (nome da feature, hash da feature, peso da feature) utilizado para preenchimento da matriz numérica que será utilizada para cômputo posterior do KNN

In [25]:
def hash_row(row):
    result = []
    for key, values in row.items():
        if key in EXCLUDED_FEATURES:
            continue
        p = FEATURE_WEIGHTS.get(key, DEFAULT_FEATURE_WEIGHT)
        tokenizer = TOKENIZERS.get(key)
        if not isinstance(values, (list, np.ndarray)):
            values = [values]
        
        for value in values:
            # handle token features
            if tokenizer:
                feature_hashes_p = hash_tokens(tokenizer, key, value, p)
                result.extend(feature_hashes_p)
                feature_hash_p = hash_feature(value, p)
                result.append(feature_hash_p)
            # handle non token features
            feature_hash_p = hash_feature(key, value)
            if feature_hash_p:
                result.append(feature_hash_p)
    return result

Segue abaixo a saída do *featurer hahser* & *shingler*. O número de features resultantes para este usuário é de 33.

Este número varia de usuário para usuário e é importante que seja utilizado uma representação esparsa da matriz de dados a serem processados.

In [26]:
row = dataset_df[dataset_df["chave_usuario"] == "U4UL"].to_dict(orient="records")[0]
print("Chave do usuário processado: " + row["chave_usuario"])
hashes = hash_row(row)
pprint(hashes, width=200)
len(hashes)

Chave do usuário processado: U4UL
[('tipo_usuario::EMPREGADO', -1143454999077322132947551267231288994892033570655, 1),
 ('centro_custo::ST75AT1B00', -1133144768198451777642148719479341277461087595357, 1),
 ('lotacao_topo::TIC', 883719330180455605187708388056895414972581655255, 1),
 ('sigla_lotacao::corp', 1195458149619987768667806989226000270421411538951, 0.2857142857142857),
 ('sigla_lotacao::dscesi', 425021426705904005561851342121354676834907806866, 0.42857142857142855),
 ('sigla_lotacao::pdds', 133469625148378281707186843726286609828382815863, 0.2857142857142857),
 ('TIC/CORP/DSCESI/DS-PDDS::3', 1133575202906053290731170417700924316681270882046, 1),
 ('sigla_lotacao::TIC/CORP/DSCESI/DS-PDDS', 580521354414209253383885614425930233875981244812, 1),
 ('nome_lotacao::desenvolvimento', -1278184407418124290890518553362925331031159140932, 0.22388059701492538),
 ('nome_lotacao::sustentacao', 200760901399190560466531582212897972524479874849, 0.16417910447761194),
 ('nome_lotacao::para', -8554

242

A função abaixo recebe uma lista de hashes e os aplica em uma linha específica de uma matriz pré-alocada.

A lista de hashes é composta tuplas no formato: (nome da feature, hash computado, peso da feature)

Em caso de hash negativo, o sinal negativo é aplicado no peso da feature

In [27]:
def apply_hashes_into(hashes, row_idx, hashed_features, feature_count):
    for feature, hash, p in hashes:
        idx = abs(hash) % feature_count
        sign = -1 if hash < 0 else 1
        hashed_features[row_idx, idx] = sign * p

No exemplo abaixo, estamos alocando uma matriz com 1 linha e HASHED_FEATURE_COUNT colunas.

O tipo np.float16 (apesar de não possuir suporte para operações diretas em ponto flutuante na CPU, somente em GPU's) é usado para economizar memória

In [28]:
rows = 1
cols = HASHED_FEATURE_COUNT
shape = rows, cols
hashed_features = np.zeros(shape, dtype=np.float16)
apply_hashes_into(hashes, row_idx=0, hashed_features=hashed_features, feature_count=HASHED_FEATURE_COUNT)
np.count_nonzero(hashed_features)

206

Aqui alocamos uma matriz esparsa do tipo "dictionary of keys" para cada um dos 85.000+ funcionários da Petrobras e com HASHED_FEATURE_COUNT colunas.

Este tipo de matriz esparsa é utilizada pois é de rápida construção iterativa. Utilizamos o tipo **float16** para redução do consumo total de memória

In [29]:
rowcount = len(dataset_df)
colcount = HASHED_FEATURE_COUNT
shape = rowcount, colcount
hashed_features = scipy.sparse.dok_array(shape, np.float16) # dictionary of keys - efficient O(1) element access
hashed_features.shape

(91798, 250000)

O trecho de código abaixo aplica a função **apply_hashes_into** na matriz esparsa pré-alocada.

Ela utiliza uma série de **generator comprehensions** para processar os dados sem que os mesmos consumam memória como uma **list comprehension** consumiria. Como o mecanismo de iteração das **generator comprehensions** é escrito em C, frequentemente este tipo de processamento apresenta uma performance melhor do que o uso de laços *for* explicitamente criados.

In [ ]:
column_names = dataset_df.columns
f = lambda row_idx, hashes: apply_hashes_into(hashes, row_idx, hashed_features, HASHED_FEATURE_COUNT)

# using generator compreehensions to save memory ... this is ugly, but uses less memory
records      = (  list(r)                     for r                   in dataset_df.itertuples(index=False) )
rows         = (  dict(zip(column_names, r))  for r                   in records                            )
hashes_rows  = (  hash_row(row)               for row                 in rows                               )
apply_hashes = (  f(row_idx, hashes_row)      for row_idx, hashes_row in enumerate(hashes_rows)             )

for i, _ in enumerate(apply_hashes):
    # for loop only used for logging purposes
    if i % 5000 == 0:
        print(f"-> {i}/{rowcount}")
print(f"-> {rowcount}/{rowcount}")

-> 0/91798
-> 5000/91798
-> 10000/91798
-> 15000/91798
-> 20000/91798
-> 25000/91798
-> 30000/91798
-> 35000/91798
-> 40000/91798
-> 45000/91798
-> 50000/91798
-> 55000/91798
-> 60000/91798


A matriz preenchida é posteriormente salva no formato **compressed sparse row**. Este formato é utilizado pois possibilita o acesso a linhas individuais de maneira otimizada (necessário para o cômputo do KNN), assim como a possibilita a execução de operações aritméticas. Este é o formato que a biblioteca **PyNNDescent** utiliza para cômputo do KNN de maneira distribuída entre as múltiplas CPU's de um computador 

In [ ]:
hashed_features = hashed_features.tocsr() # csr is fast for dot products and row slicing
scipy.sparse.save_npz(HASHED_FEATURES, hashed_features)

As informações contidas na matriz esparsa são apenas numéricas sendo necessário correlacionar uma posição da matriz com dados de um usuário. Estas informações são salvas abaixo em formato parquet para consumo posterior

In [ ]:
hashed_features_df = pd.DataFrame({
    "index"          : dataset_df.index.tolist()
,   "chave_usuario"  : dataset_df["chave_usuario"]
,   "lotacao_topo"   : dataset_df["lotacao_topo"]
,   "sigla_lotacao"  : dataset_df["sigla_lotacao"]
,   "tipo_usuario"   : dataset_df["tipo_usuario"]
,   "cargo"          : dataset_df["cargo"]
,   "enfase"         : dataset_df["enfase"]
,   "funcao"         : dataset_df["funcao"]
})

hashed_features_df.to_parquet(HASHED_FEATURES_IDX)
hashed_features_df.head(100)